```--- Day 23: Coprocessor Conflagration ---```

In [1]:
import re

In [2]:
prog = open('input.txt').readlines()


    set X Y sets register X to the value of Y.
    sub X Y decreases register X by the value of Y.
    mul X Y sets register X to the result of multiplying the value contained in register X by the value of Y.
    jnz X Y jumps with an offset of the value of Y, but only if the value of X is not zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)

In [3]:
class VM(object):
    def __init__(self, progl, prognum=0, part=1, debug=False):
        self.prog = progl[:]
        self.prognum = prognum
        self.part = part
        self.debug = debug
        self.registers = {}
        self.pc = 0
        self.running = False
        self.mul_count = 0

    def run(self):
        self.running = True
        while self.running:
            self.step()
        print(f' <P{self.prognum}> stopped')
        return self
    
    linematcher = re.compile(r'^([^ ]+) ([^ ]+)( ([^ ]+))?')

    def step(self):
        try:
            l = self.prog[self.pc].strip()
        except IndexError:
            self.running = False
            return self
        instr, op1, op2, _unused = VM.linematcher.match(l).groups()
        f, args = VM.dispatcher[instr]
        if args == 1:
            f(self, op1)
        else:
            f(self, op1, op2)
        if self.debug:
            print(f' <P{self.prognum}> {self.pc:4}: {l} | {self.registers}', flush=True)
        return self
    
    def _set(self, op1, op2):
        self.set_reg(op1, self.value(op2))
        self.pc += 1

    def _sub(self, op1, op2):
        self.set_reg(op1, self.value(op1) - self.value(op2))
        self.pc += 1
        
    def _mul(self, op1, op2):
        self.set_reg(op1, self.value(op1) * self.value(op2))
        self.mul_count += 1
        self.pc += 1
        
    def _jnz(self, op1, op2):
        if self.value(op1) != 0:
            self.pc += self.value(op2)
        else:
            self.pc += 1

    dispatcher = {'set': (_set, 2),
                  'sub': (_sub, 2),
                  'mul': (_mul, 2),
                  'jnz': (_jnz, 2)}
            
    def value(self, v):
        v = v.strip()
        try:
            return int(v)
        except ValueError:
            return self.get_reg(v)
            
    def get_reg(self, r):
        r = r.strip()
        if r in self.registers:
            return self.registers[r]
        else:
            self.registers[r] = 0
            return 0
        
    def set_reg(self, r, val):
        self.registers[r] = val

# Part 1 Actual

In [4]:
v = VM(prog)

In [5]:
v.run()

 <P0> stopped


In [6]:
v.mul_count

5929

# Part 2

## Lightly commented assembly
    start:
         0: set b 79
         1: set c b
         2: jnz a 2 # a !=0 goto alpha
         3: jnz 1 5 # goto beta
    alpha:
         4: mul b 100
         5: sub b -100000
         6: set c b
         7: sub c -17000
    beta:
         8: set f 1
         9: set d 2
    epsilon:
        10: set e 2
    delta:
        11: set g d
        12: mul g e
        13: sub g b
        14: jnz g 2 # g != 0 goto gamma
        15: set f 0
    gamma:
        16: sub e -1
        17: set g e
        18: sub g b
        19: jnz g -8 # g != 0 goto delta
        20: sub d -1
        21: set g d
        22: sub g b
        23: jnz g -13 # g != 0 goto epsilon
        24: jnz f 2 # f != 0 goto zeta
        25: sub h -1
    zeta:
        26: set g b
        27: sub g c
        28: jnz g 2 # g != 0 goto eta:
        29: jnz 1 3 # goto exit
    eta:
        30: sub b -17
        31: jnz 1 -23 # goto beta
    exit:


## translation of the assembly into Python
### Initialisation section

In [7]:
a = b = c = d = e = f = g = h = 0
a = 1 # non-debug mode
#     start:
#         set b 79
#         set c b
# debug mode
b = 79
c = b
#         jnz a 2 # a !=0 goto alpha
#         jnz 1 5 # goto beta
#     alpha:
#         mul b 100
#         sub b -100000
#         set c b
#         sub c -17000
if a != 0: # not debug mode
    b = b * 100
    b = b + 100000
    c = b
    c = c + 17000
print('initialisation:-')
print(f'a = {a}, b = {b}, c = {c}, d = {d}, e = {e}, f = {f}, g = {g}, h = {h}')

initialisation:-
a = 1, b = 107900, c = 124900, d = 0, e = 0, f = 0, g = 0, h = 0


### main program
(we don't actually execute this as it's checking naively)

In [8]:
if False:
    # python translation of assembly
    # appears to be counting which numbers in range(107900, 124900, 17) are non-prime
    while True:
        #     beta:
        #          8: set f 1
        #          9: set d 2
        f = 1
        d = 2
        while True: # for d = 2 to b
            #     epsilon:
            #         10: set e 2
            e = 2
            while True: # for e = 2 to b
                #     delta:
                #         11: set g d
                #         12: mul g e
                #         13: sub g b
                g = d * e - b
                #         14: jnz g 2 # g != 0 goto gamma
                #         15: set f 0
                if g == 0: # if d * e == b : f = 0
                    f = 0
                #     gamma:
                #         16: sub e -1
                e += 1
                #         17: set g e
                #         18: sub g b
                g = e - b
                #         19: jnz g -8 # g != 0 goto delta
                if g == 0:
                    break
            #         20: sub d -1
            d += 1
            #         21: set g d
            #         22: sub g b
            g = d - b
            #         23: jnz g -13 # g != 0 goto epsilon
            if g == 0:
                break
        #         24: jnz f 2 # f != 0 goto zeta
        #         25: sub h -1
        if f == 0:
            h += 1
        #     zeta:
        #         26: set g b
        #         27: sub g c
        g = b - c
        #         28: jnz g 2 # g != 0 goto eta:
        #         29: jnz 1 3 # goto exit
        if g == 0:
            break
        #     eta:
        #         30: sub b -17
        #         31: jnz 1 -23 # goto beta
        b = b + 17
        #     exit:


# Part 2

We need to count the non-primes in {107900, 107917, 107934, ... 124900} (increasing by 17 each time)

Approach: start with list of all numbers in the set and use Eratosthenes sieve to eliminate primes.  The number we are after is the original length (1001) minus the number of primes.

In [9]:
numbers = range(b, c+17, 17)

count_all = len(numbers)

lowest = min(numbers)

from math import sqrt

for i in range(2, int(sqrt(lowest))+1):
    numbers = [n for n in numbers if n % i != 0]

print(f'part 2 answer: {count_all - len(numbers)}')

part 2 answer: 907
